In [8]:
import sys

sys.path.append("/users/tom/git/neon_blink_detection/")
sys.path.append("/users/tom/git/neon_blink_detection/src")

from training.evaluation import evaluate
from training.helper import ClassifierParams, Results
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import cross_validate
from copy import copy
from functions.classifiers import Classifier, load_predictions, save_predictions
from src.post_processing import classify
from src.neon_blink_detector import get_params
of_params, pp_params, _ = get_params()
from src.features_calculator import create_grids
from pathlib import Path
import numpy as np
from random import choices
from training.datasets_loader import (
    concatenate,
    concatenate_all_samples,
)
from training.helper import (
    get_augmentation_options,
    get_export_dir,
    get_training_dir,
)
from src.event_array import Samples
from src.helper import OfParams, PPParams, AugParams
from training.helper import get_experiment_name_new
from training.video_loader import video_loader

clip_names = np.load("/users/tom/git/neon_blink_detection/clip_list.npy")

of_params = OfParams(n_layers=5, layer_interval=7, average=False, img_shape=(64, 64), grid_size=20, step_size=7, window_size=11, stop_steps=3)

aug_params_options = get_augmentation_options()
aug_params = aug_params_options[0]

rec = video_loader(of_params, aug_params)

rec.collect(clip_names, bg_ratio=1, augment=False, idx=0)

options 1

Loading clip: 1318-2023-02-03-10-41-29-03c72c25
Number of clips: 1

Loading clip: 1317-2023-02-03-09-41-14-30f69b87
Number of clips: 2

Loading clip: 1316-2023-02-03-09-34-09-31ef7938
Number of clips: 4

Loading clip: 1311-2023-02-02-14-30-40-96be0928
Number of clips: 1

Loading clip: 1312-2023-02-02-16-21-47-cd28e8e0
Number of clips: 1

Loading clip: 1313-2023-02-02-16-31-53-67b1bdbe
Number of clips: 1

Loading clip: 1329-2023-02-06-10-40-29-c4308424
Number of clips: 2

Loading clip: 1332-2023-02-06-11-58-03-8fc75e25
Number of clips: 1

Loading clip: 1333-2023-02-06-13-17-50-ebc1e021
Number of clips: 1

Loading clip: 1336-2023-02-06-15-28-30-adb9dbfb
Number of clips: 2

Loading clip: 1337-2023-02-06-15-36-31-211278ea
Number of clips: 1

Loading clip: 1338-2023-02-06-16-29-00-adceb23d
Number of clips: 1

Loading clip: 1339-2023-02-06-17-24-49-5b24f5ba
Number of clips: 1

Loading clip: 1340-2023-02-06-17-37-00-e6218648
Number of clips: 1

Loading clip: 1341-2023-02-07-10-38-4

In [28]:

features = concatenate(rec.all_features, clip_names)
samples_gt = concatenate_all_samples(rec.all_samples, clip_names)
labels = samples_gt.labels



In [43]:
features_l = features[:, :(features.shape[1] // 2)]
features_r = features[:, (features.shape[1] // 2):]

features_l = features_l.reshape(-1, 5, 20, 20)
features_r = features_r.reshape(-1, 5, 20, 20)

features = np.concatenate([features_l, features_r], axis=1)

(78718, 10, 20, 20)

In [44]:
X = features
y = labels

# save X and y
np.save("/users/tom/X.npy", X)
np.save("/users/tom/y.npy", y)

X.shape


(78718, 10, 20, 20)

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5)


scores_train = []
scores_test = []
i = 0

for train, test in kf.split(X):

    print(i)

    x_train, x_test = X[train], X[test]
    y_train, y_test = y[train], y[test]

    classifier = Classifier(classifier_params, export_path)
    classifier.on_fit(x_train, y_train)

    predictions = classifier.predict(x_train)
    predictions = classify(predictions, pp_params)

    scores_train.append(f1_score(y_train, predictions, average=None))

    predictions = classifier.predict(x_test)
    predictions = classify(predictions, pp_params)

    scores_test.append(f1_score(y_test, predictions, average=None))

    i += 1


In [1]:
import sys

sys.path.append("/users/tom/git/neon_blink_detection/")

import numpy as np
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.utils.data import Dataset
from training.cnn import OpticalFlowCNN, OpticalFlowDataset
import torch.optim as optim
from sklearn.model_selection import train_test_split

X = np.load("/users/tom/X.npy")
y = np.load("/users/tom/y.npy")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

model = OpticalFlowCNN()
model.train(X_train, y_train, num_epochs=10)



/cluster/anaconda3/envs/tom_py310/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
test_dataset = OpticalFlowDataset(X_test)


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [9]:
pred = model(test_dataset.X).detach().numpy()
np.argmax(pred, axis=1)

array([0, 0, 1, ..., 0, 2, 0])

In [4]:
from sklearn.metrics import f1_score
f1_score(y_test, predictions, average=None)

array([0.95631268, 0.82742065, 0.85142417])

In [14]:
import torch 
# Create a test dataset and DataLoader
test_dataset = OpticalFlowDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Set the model to evaluation mode
model.eval()

# Lists to store predictions and true labels
predicted_labels = []
true_labels = []

# Loop through the test dataset
with torch.no_grad():
    for inputs, labels in test_loader:
        # Pass the inputs to the model
        outputs = model(inputs)
        
        # Get the predicted class labels
        _, predicted = torch.max(outputs, 1)
        
        # Store the predictions and true labels
        predicted_labels.extend(predicted.numpy())
        true_labels.extend(labels.numpy())

# Calculate accuracy
accuracy = np.sum(np.array(predicted_labels) == np.array(true_labels)) / len(true_labels)
print(f"Accuracy: {accuracy:.4f}")


Accuracy: 0.9348


In [10]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [6]:
pred, true = model.predict(X_test, y_test)

TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [11]:
report_dict = classification_report(true_labels, predicted_labels, target_names=['onset', 'offset', 'background'], output_dict=True)


NameError: name 'classification_report' is not defined

In [15]:
from sklearn.metrics import classification_report

# Calculate classification metrics
report = classification_report(true_labels, predicted_labels, target_names=['onset', 'offset', 'background'])

print(report)

              precision    recall  f1-score   support

       onset       0.95      0.97      0.96     14255
      offset       0.88      0.81      0.85      1977
  background       0.89      0.85      0.87      3448

    accuracy                           0.93     19680
   macro avg       0.91      0.88      0.89     19680
weighted avg       0.93      0.93      0.93     19680



In [16]:
import sys
sys.path.append("/users/tom/git/neon_blink_detection/")
import numpy as np
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.utils.data import Dataset
from training.cnn import OpticalFlowCNN, OpticalFlowDataset
import torch.optim as optim
import torch 

def run_cnn(X, y, num_epochs=20, batch_size=32, lr=0.001, momentum=0.9):

    model = OpticalFlowCNN()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    optical_flow_data = OpticalFlowDataset(X_train, y_train)

    batch_size = 32
    train_loader = DataLoader(optical_flow_data, batch_size=batch_size, shuffle=True)

    num_epochs = 20
    for epoch in range(num_epochs):
        for i, (inputs, labels) in enumerate(train_loader, 0):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    return model



In [11]:
np.load(
                    "/users/tom/git/neon_blink_detection/clip_list.npy"
                ).shape

(20,)